# Malaria Modelling - benchmarking script 

## Programming
### Python Package Importation

This script use:
- `pandas` - data importation, manipulation.
- `seaborn` and `matplotlib` - plotting
- `numpy` - matrix calculation
- `jupyter notebook` - editing
- `black` - formatting
- `pyshp` - gis shape file
- `lxml` - xml exporting

In [289]:

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shapefile
from lxml import etree

from mbench.intervention import efficacy
from mbench import demographic
from mbench import mutil

sns.set_theme(style="whitegrid", palette="pastel")

### Create pandas dataframe for holding all data
- load gha dataframe index from PHC summary
- create columns using parameters list from Imperial model

In [ ]:
# index
gha_demographic_adm1_2020_phc = pd.read_excel(
    "../Data/GHA/Demographic/2021_PHC/2021 PHC summary.xlsx"
)
gha_demographic_adm1_2020_mean = gha_demographic_adm1_2020_phc[["region", "mean_age"]]
gha_demographic_adm1_2020_mean = demographic.reformat.adm1_name(
    df=gha_demographic_adm1_2020_mean, original_column_name="region"
)

In [ ]:
columns = [
    "eta",
    "rho",
    "a0",
    "sigma2",
    "max_age",
    "rA",
    "rT",
    "rD",
    "rU",
    "rP",
    "dE",
    "delayGam",
    "cD",
    "cT",
    "cU",
    "gamma1",
    "d1",
    "dID",
    "ID0",
    "kD",
    "uD",
    "aD",
    "fD0",
    "gammaD",
    "alphaA",
    "alphaU",
    "b0",
    "b1",
    "dB",
    "IB0",
    "kB",
    "uB",
    "phi0",
    "phi1",
    "dCA",
    "IC0",
    "kC",
    "uCA",
    "PM",
    "dCM",
    "delayMos",
    "tau1",
    "tau2",
    "mu0",
    "Q0",
    "chi",
    "bites_Bed",
    "bites_Indoors",
    "muEL",
    "muLL",
    "muPL",
    "dEL",
    "dLL",
    "dPL",
    "gammaL",
    "km",
    "cm",
    "betaL",
    "num_int",
    "itn_cov",
    "irs_cov",
    "ITN_IRS_on",
    "DY",
    "d_ITN0",
    "r_ITN0",
    "r_ITN1",
    "r_IRS0",
    "d_IRS0",
    "irs_half_life",
    "itn_half_life",
    "IRS_interval",
    "ITN_interval",
]

gha = pd.DataFrame(columns=columns, index=gha_demographic_adm1_2020_mean.index)

### Load 216 vs 260 table

From the year of xxxx, Ghana started to use the new 260 district system. From then on, the number of adm1 district increased from 10 to 16.

As some old data were generated as the old district system, the following list was generated to readjust old data to new adm1 format.

In [ ]:
district_216_260 = pd.read_csv("../Data/GHA/ADM1/216 to 260.csv")
district_216_260 = demographic.reformat.adm1_name(df=district_216_260, original_column_name='260')
print(district_216_260)
gha['old_district_name'] = district_216_260

## Ghana Admin1 District coding

### Coding

This benchmarking practices is targeting at the adm1 level of district in Ghana.

Code names of adm1 level was used for an unified experiences in the whole practices.

In [ ]:
gha_admin1 = pd.read_csv("../Data/GHA/ADM1/admin.csv")
gha_admin1 = gha_admin1[["Subdivision name", "3166-2 code"]]
gha_admin1 = gha_admin1.rename(
    columns={"3166-2 code": "code", "Subdivision name": "adm1"}
)
gha_admin1["adm1"] = gha_admin1["adm1"].map(lambda x: x.upper())
gha_admin1 = demographic.reformat.adm1_name(df=gha_admin1, original_column_name="adm1")
gha_admin1

### Adjacent ADM1 list
![image of Ghana](../Data/Ghana_Regional_Map.png)

In [ ]:
# load adjacent
gha_adjacent_provinces = pd.read_excel(
    "../Data/gha_adm_adjacent.xlsx", sheet_name="adm1_adjacent"
)
gha_adjacent_provinces = demographic.reformat.adm1_name(
    df=gha_adjacent_provinces,
    original_column_name="a",
    new_column_name="from",
    set_index=False,
)
gha_adjacent_provinces = demographic.reformat.adm1_name(
    df=gha_adjacent_provinces,
    original_column_name="b",
    new_column_name="to",
    set_index=False,
)

gha_adjacent_provinces

## Demographic Data

### Data loading
Data source
- Ghana.gov *PHC*
- us.census

The data source contains data from 1960 - 2020, datapoint of 2020 was used in this script.

### 216 vs 260 district system

Ghana switched to 260 district system in recent years. However, the old data still use 216 format. Need to pay attention.

In [ ]:
# read data and get adm1 level demographic data
gha_demographic_raw_data = pd.read_excel("../Data/GHA/demographic/ghana.xlsx", "Sheet2")
gha_demographic_adm1_raw_data = gha_demographic_raw_data.loc[
    gha_demographic_raw_data["ADM_LEVEL"] == 1
    ]
gha_demographic_adm1_raw_data = gha_demographic_adm1_raw_data.set_index(
    keys="ADM1_NAME"
)
# select the most recent 2020 data
gha_demographic_adm1_raw_data_2020 = gha_demographic_adm1_raw_data.loc[
                                     :, "B0004_2020":"B80PL_2020"
                                     ]

gha_demographic_adm1_raw_data_2020

### Median Age for adm1 district

1. Calculate total population
2. Calculate median population number
3. `.expanding` columns with `.cumsum` to get the according column
4. subtract group label using `.bfill`, `.iloc` & `.idmax`
5. get median age with `df.apply` with `lambda` function

In [ ]:
gha_demographic_adm1_2020_population_total = gha_demographic_adm1_raw_data_2020.sum(
    axis=1
)
gha_demographic_adm1_2020_population_median = (
        gha_demographic_adm1_2020_population_total / 2
)
# expand columns, each column = to cumsum the previous columns
gha_demographic_adm1_2020_population_expanding = (
    gha_demographic_adm1_raw_data_2020.expanding(axis=1).sum()
)
# expanded columns - median, the first non negative column is where median
gha_demographic_adm1_2020_raw_subtract_median = (
    gha_demographic_adm1_2020_population_expanding.subtract(
        gha_demographic_adm1_2020_population_median, axis=0
    )
)
# get ratio in groups
# check the answer here
# https://stackoverflow.com/questions/38467749/find-first-non-zero-value-in-each-row-of-pandas-dataframe/38468180
gha_demographic_adm1_2020_median_ratio = (
        gha_demographic_adm1_2020_raw_subtract_median.where(
            gha_demographic_adm1_2020_raw_subtract_median > 0
        )
        / gha_demographic_adm1_raw_data_2020
)
gha_demographic_adm1_2020_median_ratio = (
        gha_demographic_adm1_2020_median_ratio.bfill(1).iloc[:, 0] * 4
)
# get group labels
gha_demographic_adm1_2020_median_group_value = (
        gha_demographic_adm1_raw_data_2020 - gha_demographic_adm1_2020_population_expanding
)
gha_demographic_adm1_median_group_labels = (
        gha_demographic_adm1_2020_raw_subtract_median > 0
).idxmax(axis=1)
# get first two string as the start of age group
gha_demographic_adm1_median_group_labels = (
    gha_demographic_adm1_median_group_labels.apply(lambda x: float(x[1:3]))
)
# add age group with ratio
gha_demographic_adm1_2020_median = (
        gha_demographic_adm1_median_group_labels + gha_demographic_adm1_2020_median_ratio
)

gha_demographic_adm1_2020_median

gha_demographic_adm1_2020_mean

# plotting
gha_demographic_adm1_2020_mean.plot.bar()

### Adm2 level population
Data source: GHA/incid from WHO/GMP

In [ ]:
incid = pd.read_csv("../Data/GHA/Routine_data/District-level/incid.csv")
incid = incid[incid["year"] == 2018]
incid = demographic.reformat.adm1_name(df=incid, original_column_name="adm1", set_index=False)
incid.sample(5)
gha_adm2_population = incid[["adm1", "adm2", "pop"]]
gha_adm2_population = gha_adm2_population.rename(columns={"pop": "population"})
gha_adm2_population.sample(25)

### Exponential Population Distribution - ETA

In the malaria models, exponential distribution was used to simulation the age structure.

Estimation of the **ETA** parameter: death rates for exponential population distribution,

$$ ETA = \frac{1}{MeanAge} $$

Default ETA = 0.0001305

In [ ]:
gha["eta"] = 1 / (gha_demographic_adm1_2020_mean * 365)
gha["eta"]

### Max age
@param max_age Maximum age in days. Default = 100*365

In [ ]:
gha["max_age"] = 100 * 365

In [ ]:
#pd.merge(
#    gha,
#
#)
#xml_demography = etree.Element("demography")
#xml_demography.set("maximumAgeYrs", 90)
#xml_demography.set("name", )
#xml_demography.append(etree.Element())

## State transition related parameters

@param rA Rate of leaving asymptomatic infection. Default = 0.00512821

@param rT Rate of leaving treatment. Default = 0.2

@param rD Rate of leaving clinical disease. Default = 0.2

@param rU Rate of recovering from subpatent infection. Default = 0.00906627

@param rP Rate of leaving prophylaxis. Default = 0.06666667

@param dE Latent period of human infection. Default = 12

@param delayGam Lag from parasites to infectious gametocytes. Default = 12.5

@param cD Untreated disease contribution to infectiousness. Default = 0.0676909

@param cT Treated disease contribution to infectiousness. Default =   0.322 * cD

@param cU Subpatent disease contribution to infectiousness. Default = 0.006203

@param gamma1 Parameter for infectiousness of state A. Default = 1.82425

In [ ]:
gha["rA"] = 0.00512821
gha["rT"] = 0.2
gha["rD"] = 0.2
gha["rU"] = 0.00906627
gha["rP"] = 0.06666667
# TODO check if this data is available from the data folder
gha["dE"] = 12
gha["delayGam"] = 12.5
gha["cD"] = 0.0676909
gha["cT"] = 0.322 * gha["cD"]
gha["cU"] = 0.006203
gha["gamma1"] = 1.82425

## Immunity parameters
@param d1 Minimum probability due to maximum immunity. Default = 0.160527

@param dID Inverse of decay rate. Default = 3650

@param ID0 Scale parameter. Default = 1.577533

@param kD Shape parameter. Default = 0.476614

@param uD Duration in which immunity is not boosted. Default = 9.44512

@param aD Scale parameter relating age to immunity. Default = 8001.99

@param fD0 Time-scale at which immunity changes with age. Default = 0.007055

@param gammaD Shape parameter relating age to immunity. Default = 4.8183

@param alphaA PCR detection probability parameters state A. Default = 0.757

@param alphaU PCR detection probability parameters state U. Default = 0.186

@param b0 Maximum probability due to no immunity. Default = 0.590076

@param b1 Maximum relative reduction due to immunity. Default = 0.5

@param dB Inverse of decay rate. Default = 3650

@param IB0 Scale parameter. Default = 43.8787

@param kB Shape parameter. Default = 2.15506

@param uB Duration in which immunity is not boosted. Default = 7.19919

@param phi0 Maximum probability due to no immunity. Default = 0.791666

@param phi1 Maximum relative reduction due to immunity. Default = 0.000737

@param dCA Inverse of decay rate. Default = 10950

@param IC0 Scale parameter. Default = 18.02366

@param kC Shape parameter. Default = 2.36949

@param uCA Duration in which immunity is not boosted. Default = 6.06349

@param PM New-born immunity relative to mother’s. Default = 0.774368

@param dCM Inverse of decay rate of maternal immunity. Default = 67.6952

In [ ]:
gha["d1"] = 0.160527
gha["dID"] = 3650
gha["ID0"] = 1.577533
gha["kD"] = 0.476614
gha["uD"] = 9.44512
gha["aD"] = 8001.99
gha["fD0"] = 0.007055
gha["gammaD"] = 4.8183
gha["alphaA"] = 0.757
gha["alphaU"] = 0.186
gha["b0"] = 0.590076
# Maximum relative reduction due to immunity
gha["b1"] = 0.5
# Immunity Inverse of decay rate
gha["dB"] = 3650
gha["IB0"] = 43.8787
gha["kB"] = 2.15506
gha["uB"] = 7.19919
gha["phi0"] = 0.791666
gha["phi1"] = 0.000737
gha["dCA"] = 10950
gha["IC0"] = 18.02366
gha["kC"] = 2.36949
gha["uCA"] = 6.06349
gha["PM"] = 0.774368
gha["dCM"] = 67.695

## Vector
### Extrinsic incubation period
@param delayMos Extrinsic incubation period. Default = 10

In [ ]:
# TODO check this parameter, extrinsic incubation period
gha["delayMos"] = 10

### Biting parameters
#### Host seeking
@param tau1 Duration of host seeking, assumed to be constant between species. Default = 0.69

#### Duration of resting after feed
@param tau2 Duration of mosquito resting after feed. Default = 2.31

#### Daily mortality rate
@param mu0 Daily mortality of adult mosquitos. Default = 0.132

> TODO use data from Ellie's paper and WHO/GMP data to estimation provincial level *anthrophagy* and *endophily*

#### Anthrophagy
@param Q0 Anthrophagy probability. Default = 0.92

#### Endophily
@param chi Endophily probability. Default = 0.86

#### Bites percentage both indoors and in bed
@param bites_Bed Percentage of bites indoors and in bed. Default = 0.89

#### Bites percentage indoors
@param bites_Indoors Percentage of bites indoors . Default = 0.97

In [ ]:
# duration of host seeking, assumed to be constant between species
gha["tau1"] = 0.69
# duration of mosquito resting after feed
gha["tau2"] = 2.31
# daily mortality of adult mosquitos
# TODO
gha["mu0"] = 0.13
# Anthrophagy probability
# TODO
gha["Q0"] = 0.92
# TODO
# Endophily probability
gha["mu0"] = 0.86
# TODO
# Percentage of bites indoors and in bed
gha["bites_Bed"] = 0.89
# Percentage of bites indoors
gha["bites_Indoors"] = 0.97

#### Biting rates
@param rho Age-dependent biting parameter. Default = 0.85

@param a0 Age-dependent biting parameter. Default = 2920

@param sigma2 Variance of the log heterogeneity in biting rates. Default = 1.67

In [ ]:
gha["rho"] = 0.85
gha["a0"] = 2920
gha["sigma2"] = 1.67

### Larvae and Pupae
@param muEL Per capita daily mortality rate of early stage larvae (low density). Default = 0.0338

@param muLL Per capita daily mortality rate of late stage larvae (low density). Default = 0.0348

@param muPL Per capita daily mortality rate of pupae. Default = 0.249

@param dEL Development time of early stage larvae. Default = 6.64

@param dLL Development time of late stage larvae. Default = 3.72

@param dPL Development time of pupae. Default = 0.643

@param gammaL Relative effect of density dependence on late instars relative to early instars. Default = 13.25

In [ ]:
# Per capita daily mortality rate of early stage larvae(low density)
gha["muEL"] = 0.0338
# Per capita daily mortality rate of late stage larvae(low density)
gha["muLL"] = 0.0348
# Per capita daily mortality rate of pupae
gha["muPL"] = 0.249
# Development time of early stage larvae
gha["dEL"] = 6.64
# Development time of late stage larvae
gha["dLL"] = 3.72
# Development time of pupae
gha["dPL"] = 0.643
# Relative effect of density dependence on late instars relative to early instars
gha["gammaL"] = 13.25

## Seasonal parameters
@param km Seasonal carrying capacity. Default = 11

@param cm Seasonal birth rate. Default = 0.05

@param betaL Number of eggs laid per day per mosquito. Default = 21.2

### Imperial Model Implementation

Fourier's transformation, all the seasonal parameters have been precalculated and provided in their github [repository](https://github.com/mrc-ide/deterministic-malaria-model/blob/master/data-raw/admin_units_seasonal.csv)

By providing the country name and adm1 level district name, the programme will search for the seasonal parameters.

But one problem for Ghana is that they use **216 district**.

The problem was solved by using 216 to 260 district mapping.

In [ ]:
# Seasonal carrying capacity
gha["km"] = 11
# Seasonal birth rate
gha["cm"] = 0.05
# N of eggs laid per day per mosquito
gha["betaL"] = 21.2

## Interventions
### Usage
@param num_int Number of intervention parameters.  Default = 4

@param itn_cov The proportion of people that use an ITN. Default = 0

@param irs_cov The proportion of people living in houses that have been sprayed. Default = 0

@param ITN_IRS_on Time of ITN and IRS to be activated. Default = -1, i.e. never.

@param DY Duration of year (days). Default = 365

Data source - GHA_new_district_summaries - WHO/GMP

aggregation over

$$ITN_{adm1} =\frac{\sum{(P_{adm2}*ITN_{adm2})}}{P_{adm1}}$$

$$IRS_{adm1} =\frac{\sum{(P_{adm2}*IRS_{adm2})}}{P_{adm1}}$$

In [ ]:
# N of intervention parameters
gha["num_int"] = 2

In [ ]:
# Time of ITN and IRS to be activated.
gha["ITN_IRS_on"] = 200
# Duration of years
gha["DY"] = 365

In [ ]:
# gha_intervention = pd.read_stata('../Data/GHA/interventions/interventions.dta')
intervention = pd.read_excel(
    io="/Users/sepmein/Dropbox/benchmarking/Data/GHA/MAP_District_Estimates/Maps_by_MAP_260districts/GHA_summaries/GHA_new_district_summaries.xlsx",
    sheet_name="district_summaries",
)
intervention = demographic.reformat.adm1_name(
    df=intervention, original_column_name="REGION", set_index=False
)

intervention = intervention.rename(columns={"DISTRICT": "adm2"})
itn_coverage = intervention[["adm1", "adm2", "ITN_2018"]]
itn_coverage = pd.merge(
    left=gha_adm2_population, right=itn_coverage, left_on="adm2", right_on="adm2"
)

itn_coverage = itn_coverage.drop(columns=["adm1_x"], axis=1)
itn_coverage = itn_coverage.rename(columns={"adm1_y": "adm1"})
itn_coverage["population_itn"] = itn_coverage["population"] * itn_coverage["ITN_2018"]
itn_coverage_groupby_sum = itn_coverage.groupby(by="adm1").sum()
itn_coverage_groupby_sum["itn_coverage"] = (
        itn_coverage_groupby_sum["population_itn"] / itn_coverage_groupby_sum["population"]
)
gha["itn_cov"] = itn_coverage_groupby_sum["itn_coverage"]
sns.stripplot(gha["itn_cov"], size=7, color=".26")
sns.boxplot(gha["itn_cov"], showfliers=False)

In [ ]:
irs_coverage = intervention[["adm1", "adm2", "IRS_2018"]]
irs_coverage = pd.merge(
    left=gha_adm2_population, right=irs_coverage, left_on="adm2", right_on="adm2"
)

irs_coverage = irs_coverage.drop(columns=["adm1_x"], axis=1)
irs_coverage = irs_coverage.rename(columns={"adm1_y": "adm1"})
irs_coverage["population_irs"] = irs_coverage["population"] * irs_coverage["IRS_2018"]
irs_coverage_groupby_sum = irs_coverage.groupby(by="adm1").sum()
irs_coverage_groupby_sum["irs_coverage"] = (
        irs_coverage_groupby_sum["population_irs"] / irs_coverage_groupby_sum["population"]
)
gha["irs_cov"] = irs_coverage_groupby_sum["irs_coverage"]
sns.stripplot(gha["irs_cov"], size=7, color=".26")
sns.boxplot(gha["irs_cov"], showfliers=False)
gha["irs_cov"]

### Efficacy
@param d_ITN0 Probability of dying with an encounter with ITN (max). Default = 0.41

@param r_ITN0 Probability of repeating behaviour with ITN (max). Default = 0.56

@param r_ITN1 Probability of repeating behaviour with ITN (min). Default = 0.24

@param r_IRS0 Probability of repeating behaviour with IRS (min). Default = 0.6

@param d_IRS0 Probability of dying with an encounter with IRS (max). Default = 1

In [ ]:
for x in np.nditer(np.linspace(0, 1, 20)):
    y = efficacy.converter.bioassay_to_rds(x)
    print('regular, repeating: ', "{:10.2f}".format(y[0][0]))
    print("regular, repeating, with decay:", "{:10.2f}".format(y[0][1]))
    print("regular, dying: ", "{:10.2f}".format(y[0][2]))
    print("regular, feeding: ", "{:10.2f}".format(y[0][3]))
    print("PBO, repeating: ", "{:10.2f}".format(y[1][0]))
    print("PBO, repeating, with decay:", "{:10.2f}".format(y[1][1]))
    print("PBO, dying: ", "{:10.2f}".format(y[1][2]))
    print("PBO, feeding: ", "{:10.2f}".format(y[1][3]))

In [ ]:
# Probability of dying with an encounter with ITN(max)
gha["d_ITN0"] = 0.41

# load mortality settings vs LLINs & PBO nets
mortality_llins_vs_pbo = pd.read_csv("../Data/LLINs vs PBO/mortality rate llins vs pbo.csv")
# load resistance settings in each adm1
gha_adm1_vector_resistance = pd.read_excel(
    '../Data/GHA/WHO_IR_data/MTM_DISCRIMINATING_CONCENTRATION_BIOASSAY_20211130.xlsx',
    sheet_name='Data'
)
# rename Brong Ahafo to Brong-Ahafo
gha_adm1_vector_resistance = gha_adm1_vector_resistance.replace('Brong Ahafo', 'Brong-Ahafo')

# subgroup
gha_adm1_vector_resistance = gha_adm1_vector_resistance.groupby(
    'ADMIN1'
).mean()

gha_adm1_vector_resistance = gha_adm1_vector_resistance['MORTALITY_ADJUSTED']
gha = gha.merge(right=gha_adm1_vector_resistance,
                how="left",
                left_on="old_district_name",
                right_index=True
                )
gha['r_ITN_0'] = gha['MORTALITY_ADJUSTED'].map(lambda x: efficacy.converter.bioassay_to_rds(x / 100)[0][0])
gha['r_PBO_0'] = gha['MORTALITY_ADJUSTED'].map(lambda x: efficacy.converter.bioassay_to_rds(x / 100)[1][0])
gha['r_ITN_1'] = gha['MORTALITY_ADJUSTED'].map(lambda x: efficacy.converter.bioassay_to_rds(x / 100)[0][1])
gha['r_PBO_1'] = gha['MORTALITY_ADJUSTED'].map(lambda x: efficacy.converter.bioassay_to_rds(x / 100)[1][1])
gha['d_ITN_0'] = gha['MORTALITY_ADJUSTED'].map(lambda x: efficacy.converter.bioassay_to_rds(x / 100)[0][2])
gha['d_PBO_0'] = gha['MORTALITY_ADJUSTED'].map(lambda x: efficacy.converter.bioassay_to_rds(x / 100)[1][2])
# categorize resistance into h/m/l groups
# gha_adm1_vector_resistance_groups = pd.cut(
#     x=gha_adm1_vector_resistance,
#     bins=[0.0, 30, 60, 100],
#     labels=['high', 'moderate', 'low']
# )
# as the data from WHO/GMP/Malaria Threats Map still use 216 system
# I need to transform to the 260 system first
# gha['']

# Probability of repeating behaviour with ITN(max)
# gha["r_ITN0"] = 0.56
# Probability of repeating behaviour with ITN(min)
# gha["r_ITN1"] = 0.24
# Probability of repeating behaviour with IRS(min)
gha["r_IRS0"] = 0.6
# Probability of dying with an encounter with IRS(max)
gha["d_IRS0"] = 1

In [ ]:
# gha = gha.merge(right=gha_adm1_vector_resistance_groups,
#                 how="left",
#                 left_on="old_district_name",
#                 right_index=True
#                 )
# gha.loc[gha['MORTALITY_ADJUSTED'] == 'low', 'd_ITN0_LLINs'] = .527
# gha.loc[gha['MORTALITY_ADJUSTED'] == 'low', 'd_ITN0_PBO'] = .659
# gha.loc[gha['MORTALITY_ADJUSTED'] == 'moderate', 'd_ITN0_LLINs'] = .18
# gha.loc[gha['MORTALITY_ADJUSTED'] == 'moderate', 'd_ITN0_PBO'] = .303
# gha.loc[gha['MORTALITY_ADJUSTED'] == 'high', 'd_ITN0_LLINs'] = .238
# gha.loc[gha['MORTALITY_ADJUSTED'] == 'high', 'd_ITN0_PBO'] = .438

### Decay
@param irs_half_life IRS half life. Default =   0.5 * DY

@param itn_half_life ITN half life. Default =   2.64 * DY

In [ ]:
# IRS half life
gha["irs_half_life"] = 0.5 * 365
# ITN half life
gha["itn_half_life"] = 2.64 * 365

### Implementation
@param IRS_interval How long before IRS is repeated, i.e. when IRS decay = 1. Default =   1 * DY

@param ITN_interval How long before ITN is repeated, i.e. when IRS decay = 1.  Default =   3 * DY

In [ ]:
# How long before IRS is repeated
gha["IRS_interval"] = 1 * 365
# how long before ITN is repeated
gha["ITN_interval"] = 3 * 365

In [ ]:
GHA_Intervention_data = pd.read_excel(
    "../Data/GHA/Interventions/GHA_Intervention_data.xlsx", "Data template "
)
GHA_Intervention_data

In [ ]:
pd.merge(
    gha_admin1,
    GHA_Intervention_data,
    left_on="adm1",
    right_on="adm1",
)

## EIR estimation

Data source: literature search

- Searched with the keyword 'eir', 'entomology inoculation rate', 'Ghana'
- Created a database on EIR for about 800 records

In [ ]:
eir = pd.read_csv("../Data/eir.csv")
ghana_eir = eir[eir["Country"] == "Ghana"]

In [ ]:
sns.boxplot(data=eir, y="eir", x="setting", showfliers=False)

In [ ]:
eir["Year_Catagorical"] = eir["Year"] // 10 * 10
sns.boxenplot(data=eir, y="eir", x="Year_Catagorical", showfliers=False)

In [ ]:
# plot ghana eir data
# TODO: To include 95%CI low and high data.
g = sns.swarmplot(data=ghana_eir, y="eir", x="adm1")
plt.xticks(rotation=45)

### Ghana EIR calculation and missing provinces data estimation

1. search through papers to find eir dataset
2. generate existing dataset
3. check availability for adm1
4. calculate eir for available adm1 region
5. estimate eir for unavailable adm1 region

In [ ]:
gha_eir = ghana_eir.groupby(by="adm1").mean()
gha_eir = gha_eir["eir"]
gha_eir = demographic.reformat.adm1_name(gha_eir)
gha_eir
gha["eir"] = gha_eir

gha = util.interpolate.missing_data(
    df=gha, neighbour=gha_adjacent_provinces, column="eir", round_n=10
)

gha["eir"].plot.bar()

## Treatment seeking

Data source - WHO/GMP
Latest data - 2016

- **any_treat** - percentage of treatment from public sector **and** private sector
- **pub_treat** - data from **only** public sector

In [ ]:
# 216 region
treatment_seeking_216 = pd.read_csv(
    "../Data/GHA/MAP_District_Estimates/Maps_byMAP_216districts/GHA_regional_DHS_treatment_seeking/GHA_TreatmentSeeking.csv"
)
treatment_seeking_216 = treatment_seeking_216.rename(columns={"REGNAME": "adm1"})
treatment_seeking_216 = treatment_seeking_216[["adm1", "any_treat", "pub_treat"]]

ax = treatment_seeking_216.plot(kind="bar")
##ax.set_xticks(treatment_seeking_216[['adm1']])
ax.set_xticklabels(treatment_seeking_216["adm1"])

##treatment_seeking_216[['adm1']]
treatment_seeking_216

In [ ]:
# 260 region

treatment_seeking_260 = pd.read_excel(
    "../Data/GHA/MIS-DHS_data/GHA_treatment_seeking_tprs_adm1_16regions.xlsx", "Sheet"
)
treatment_seeking_260 = treatment_seeking_260[["adm1", "pub_treat"]]
treatment_seeking_260 = treatment_seeking_260.rename(
    columns={"pub_treat": "treatment_seeking"}
)

treatment_seeking_260 = demographic.reformat.adm1_name(
    df=treatment_seeking_260, original_column_name="adm1"
)
treatment_seeking_260

ax = treatment_seeking_260.plot(kind="bar")
##ax.set_xticks(treatment_seeking_216[['adm1']])

In [ ]:
gha = pd.merge(
    left=gha, right=treatment_seeking_260, how="left", left_index=True, right_index=True
)
gha["treatment_seeking"]

# Map ploting with Shapefile in PYTHON

In [ ]:
# shapefile
gha_shape = shapefile.Reader(
    "../Data/GHA/WHO_Health_District_Shapefiles/Ghana _Shape_260 Districts_Boundary. 26.06.2019/260_Districts_of_Ghana"
)
shapes = gha_shape.shapes()

In [ ]:
# for shape in shapes:
#     for point in shape.points:
#         x,y = point
#         plt.plot(x,y)
# plt.show()

In [ ]:
gha_shape.records()


def read_shapefile(sf):
    """
    Read a shapefile into a Pandas dataframe with a 'coords'
    column holding the geometry information. This uses the pyshp
    package
    """
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df


gha_shape_df = read_shapefile(gha_shape)
gha_shape_df.shape
gha_shape_df.sample(5)

In [ ]:
def plot_shape(id, s=None, sf=None):
    """PLOTS A SINGLE SHAPE"""
    plt.figure()
    ax = plt.axes()
    ax.set_aspect("equal")
    shape_ex = sf.shape(id)
    x_lon = np.zeros((len(shape_ex.points), 1))
    y_lat = np.zeros((len(shape_ex.points), 1))
    for ip in range(len(shape_ex.points)):
        x_lon[ip] = shape_ex.points[ip][0]
        y_lat[ip] = shape_ex.points[ip][1]
    plt.plot(x_lon, y_lat)
    x0 = np.mean(x_lon)
    y0 = np.mean(y_lat)
    plt.text(x0, y0, s, fontsize=10)
    # use bbox (bounding box) to set plot limits
    plt.xlim(shape_ex.bbox[0], shape_ex.bbox[2])
    return x0, y0


def plot_map(sf, x_lim=None, y_lim=None, figsize=(11, 9)):
    """
    Plot map with lim coordinates
    """
    plt.figure(figsize=figsize)
    id = 0
    for shape in sf.shapeRecords():
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        plt.plot(x, y, "k")

        # if (x_lim == None) & (y_lim == None):
        #    x0 = np.mean(x)
        #    y0 = np.mean(y)
        #    plt.text(x0, y0, id, fontsize=10)
        # id = id+1

    if (x_lim != None) & (y_lim != None):
        plt.xlim(x_lim)
        plt.ylim(y_lim)


plot_map(gha_shape, figsize=(10, 12))

In [ ]:
# Export to models

In [ ]:
gha.to_csv("../Data/GHA/gha.csv")


In [ ]:
root = etree.Element('root')
root.append(etree.Element('child1'))
root.tag